In [20]:
import pandas as pd 
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from collections import Counter
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/emma-raducanu/emma_tweets.csv
/kaggle/input/emma-raducanu/tennis_ball.png


In [21]:
tweets = pd.read_csv('../input/emma-raducanu/emma_tweets.csv')
tweets.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1436825934519209986,Bex Lynch,"Carlisle, England",girl. northern.,2010-10-20 19:10:14,117,426,1515,False,2021-09-11 22:56:12,18 year old female relegates the mega-hype aro...,"['ChristianoWho', 'EmmaRaducanu']",Twitter for Android,0,0,False
1,1436825880861478914,Rupinder K Gill,"England, United Kingdom","Here shining a light on NHS, womens rights, cl...",2012-05-07 10:28:26,2635,5001,13070,False,2021-09-11 22:55:59,She's done it! #EmmaRaducanu is #USOpen champi...,"['EmmaRaducanu', 'USOpen']",Twitter for Android,0,0,False
2,1436825829229666306,𝚕𝚊𝚞𝚛𝚎𝚗 ⋆,NaN,spend my life screaming at the tv when tennis ...,2011-04-20 17:57:55,1646,685,94663,False,2021-09-11 22:55:47,me for the foreseeable future.\n#EmmaRaducanu ...,"['EmmaRaducanu', 'USOpen']",Twitter for iPhone,0,0,False
3,1436825771624988676,Neeraj Nanaiah,"Mysore, India","Sports fan, Gooner.",2019-06-29 15:32:01,134,159,10035,False,2021-09-11 22:55:33,What a tennis story! Still can't wrap it aroun...,"['EmmaRaducanu', 'USOpen']",Twitter for Android,0,0,False
4,1436825431362179073,alastair mciver,"South East, England",Writer; Founder - GoWriteAhead website; featur...,2010-07-13 10:55:43,157,211,5397,False,2021-09-11 22:54:12,Rather overshadows the incredible doubles' dou...,NaN,Twitter for Android,0,0,False


In [22]:
tweets.shape
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10485 entries, 0 to 10484
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                10485 non-null  int64 
 1   user_name         10485 non-null  object
 2   user_location     8160 non-null   object
 3   user_description  9673 non-null   object
 4   user_created      10485 non-null  object
 5   user_followers    10485 non-null  int64 
 6   user_friends      10485 non-null  int64 
 7   user_favourites   10485 non-null  int64 
 8   user_verified     10485 non-null  bool  
 9   date              10485 non-null  object
 10  text              10485 non-null  object
 11  hashtags          7932 non-null   object
 12  source            10485 non-null  object
 13  retweets          10485 non-null  int64 
 14  favorites         10485 non-null  int64 
 15  is_retweet        10485 non-null  bool  
dtypes: bool(2), int64(6), object(8)
memory usage: 1.1+ MB


In [23]:
#Preprocessing

def preprocess(text):
    text=str(text)
    text = text.lower()
    text=text.replace('{html}',"") 
    cleantxt = re.compile('<.*?>')
    cleantext = re.sub(cleantxt, '', text)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


tweets['cleanText']=tweets['text'].map(lambda s:preprocess(s)) 

In [24]:
tweets.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,cleanText
0,1436825934519209986,Bex Lynch,"Carlisle, England",girl. northern.,2010-10-20 19:10:14,117,426,1515,False,2021-09-11 22:56:12,18 year old female relegates the mega-hype aro...,"['ChristianoWho', 'EmmaRaducanu']",Twitter for Android,0,0,False,year old female relegates mega hype around bac...
1,1436825880861478914,Rupinder K Gill,"England, United Kingdom","Here shining a light on NHS, womens rights, cl...",2012-05-07 10:28:26,2635,5001,13070,False,2021-09-11 22:55:59,She's done it! #EmmaRaducanu is #USOpen champi...,"['EmmaRaducanu', 'USOpen']",Twitter for Android,0,0,False,done emmaraducanu usopen champion debut well d...
2,1436825829229666306,𝚕𝚊𝚞𝚛𝚎𝚗 ⋆,NaN,spend my life screaming at the tv when tennis ...,2011-04-20 17:57:55,1646,685,94663,False,2021-09-11 22:55:47,me for the foreseeable future.\n#EmmaRaducanu ...,"['EmmaRaducanu', 'USOpen']",Twitter for iPhone,0,0,False,foreseeable future emmaraducanu usopen
3,1436825771624988676,Neeraj Nanaiah,"Mysore, India","Sports fan, Gooner.",2019-06-29 15:32:01,134,159,10035,False,2021-09-11 22:55:33,What a tennis story! Still can't wrap it aroun...,"['EmmaRaducanu', 'USOpen']",Twitter for Android,0,0,False,tennis story still wrap around head emmaraduca...
4,1436825431362179073,alastair mciver,"South East, England",Writer; Founder - GoWriteAhead website; featur...,2010-07-13 10:55:43,157,211,5397,False,2021-09-11 22:54:12,Rather overshadows the incredible doubles' dou...,NaN,Twitter for Android,0,0,False,rather overshadows incredible doubles double a...


In [25]:
#Most common words
tweets['Counter'] = tweets['cleanText'].apply(lambda x:str(x).split())
top = Counter([item for sublist in tweets['Counter'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Greys')

,Common_words,count
0,emmaraducanu,8642
1,usopen,2943
2,emma,1514
3,tennis,1397
4,raducanu,1150
5,open,891
6,congratulations,872
7,win,834
8,well,789
9,amazing,741
